# Paperpile to Notion

[API Reference](https://developers.notion.com/)

`% conda activate api`

`% conda install -c conda-forge jsonpath-ng`


In [ ]:
import requests
import secret
import json

In [ ]:
# Retrieve a database

base_url = 'https://api.notion.com/v1/databases/'

secret.NOTION_API_KEY = secret.NOTION_API

header = {'Authorization': secret.NOTION_API_KEY,
          'Notion-Version': '2021-08-16', 'Content-Type': 'application/json'}
query = {'filter': {'property': 'Sync', 'checkbox': {'equals': True}}}
query = {}

response = requests.post(base_url + secret.NOTION_DATABASE_ID + '/query',
                         headers=header, json=query)
response


In [ ]:
print(secret)

In [ ]:
response.json()

In [ ]:
# Create a Page

base_url = 'https://api.notion.com/v1/pages'

header = {'Authorization': secret.NOTION_API_KEY,
          'Notion-Version': '2021-08-16', 'Content-Type': 'application/json'}

query = {
    'parent': {'database_id': secret.NOTION_DATABASE_ID},
    'properties': {
        'Number': 401,
        'Name': [{'type': 'text', 'text': {'content': 'from Python'}}],
        'Column': [{'type': 'text', 'text': {'content': "Tag 1"}}],
    }
}

response = requests.post(base_url, headers=header, json=query)
response


In [ ]:
# Paperpile JSON

fn = ('data/Paperpile - Oct 06 JSON Export.txt')
with open(fn) as f:
    papers = json.load(f)

title = papers[-1]['title']
url = papers[-1]['url']
doi = papers[-1]['doi']
published = papers[-1]['published']['year']
abstract = papers[-1]['abstract']
author = papers[-1]['author']
(title, url, doi, published, abstract, author)


In [ ]:
def decode(x):
    return [i for i in x]

decode(papers[-1])




In [ ]:
# Create a Page

base_url = 'https://api.notion.com/v1/pages'

header = {'Authorization': secret.NOTION_API_KEY,
          'Notion-Version': '2021-08-16', 'Content-Type': 'application/json'}

query = {
    'parent': {'database_id': secret.NOTION_DATABASE_ID},
    'properties': {
        'Title': [{'type': 'text', 'text': {'content': title}}],
        # 'Author': [{'type': 'text', 'text': {'content': author[0]['bak']}}],
        'Published': [{'type': 'text', 'text': {'content': published}}],
        'doi': [{'type': 'text', 'text': {'content': doi}}],
        # 'URL': {'url': 'url[0]'}
    }
}


query = {
    'parent': {'database_id': secret.NOTION_DATABASE_ID},
    'properties': {
        'Title': [{'type': 'text', 'text': {'content': title}}],
        'Published': [{'type': 'text', 'text': {'content': published}}],
        'doi': [{'type': 'text', 'text': {'content': doi}}],
    }
}

response = requests.post(base_url, headers=header, json=query)
response


In [ ]:
response.json()

In [55]:
def propTitle(_text):
    return {"title": [{"type": "text", "text": {"content": _text}}]}

def propRichText(_text):
    return {"rich_text": [{"type": "text", "text": {"content": _text}}]}


def propText(_text):
    return {"text": [{"type": "text", "text": {"content": _text}}]}

def propNumber(_number):
    return {"number": _number}

def propURL(_url):
    return {"url": _url}


In [56]:
paragraph = (dict(
    object='block',
    type='paragraph',
    paragraph=propText(abstract)))

paragraph

children = []
children.append(paragraph)

children


[{'object': 'block',
  'type': 'paragraph',
  'paragraph': {'text': [{'type': 'text',
     'text': {'content': "Research indicates that the extent to which one becomes engaged, transported, or immersed in a narrative influences the narrative's potential to affect subsequent story-related attitudes and beliefs. Explaining narrative effects and understanding the mechanisms responsible depends on our ability to measure narrative engagement in a theoretically meaningful way. This article develops a scale for measuring narrative engagement that is based on a mental models approach to narrative processing. It distinguishes among four dimensions of experiential engagement in narratives: narrative understanding, attentional focus, emotional engagement, and narrative presence. The scale is developed and validated through exploratory and confirmatory factor analyses with data from viewers of feature film and television, in different viewing situations, and from two different countries. The scale

In [63]:
# Paperpile JSON

fn = ('data/Paperpile - Oct 06 JSON Export.txt')
with open(fn) as f:
    papers = json.load(f)

idx = 199

title = papers[idx]['title']
url = papers[idx]['url']
doi = papers[idx]['doi']
published = papers[idx]['published']['year']
abstract = papers[idx]['abstract']
author = papers[idx]['author']

# (title, url, doi, published, abstract, author)

# Create a Page

base_url = 'https://api.notion.com/v1/pages'

header = {'Authorization': secret.NOTION_API_KEY,
          'Notion-Version': '2021-08-16', 'Content-Type': 'application/json'}

query = (dict(
    parent=dict(database_id=secret.NOTION_DATABASE_ID),
    properties=dict(
        Title=propTitle(title),
        Published=propRichText(published),
        doi=propRichText(doi),
        URL=propURL(url[0])
    ),
    children=children))

print(query)
response = requests.post(base_url, headers=header, json=query)
response


{'parent': {'database_id': 'c342e37eb74f4b068664588c0de7cac3'}, 'properties': {'Title': {'title': [{'type': 'text', 'text': {'content': 'Visualization for Decision Support in FITradeoff Method: Exploring Its Evaluation with Cognitive Neuroscience'}}]}, 'Published': {'rich_text': [{'type': 'text', 'text': {'content': '2017'}}]}, 'doi': {'rich_text': [{'type': 'text', 'text': {'content': '10.1007/978-3-319-57487-5_5'}}]}, 'URL': {'url': 'http://dx.doi.org/10.1007/978-3-319-57487-5_5'}}, 'children': [{'object': 'block', 'type': 'paragraph', 'paragraph': {'text': [{'type': 'text', 'text': {'content': "Research indicates that the extent to which one becomes engaged, transported, or immersed in a narrative influences the narrative's potential to affect subsequent story-related attitudes and beliefs. Explaining narrative effects and understanding the mechanisms responsible depends on our ability to measure narrative engagement in a theoretically meaningful way. This article develops a scale f

<Response [200]>

In [58]:
response.json()

{'object': 'page',
 'id': 'a3f2691c-641c-44e8-82a5-d124e19065fa',
 'created_time': '2021-10-07T23:39:00.000Z',
 'last_edited_time': '2021-10-07T23:39:00.000Z',
 'cover': None,
 'icon': None,
 'parent': {'type': 'database_id',
  'database_id': 'c342e37e-b74f-4b06-8664-588c0de7cac3'},
 'archived': False,
 'properties': {'Published': {'id': 'JZQS',
   'type': 'rich_text',
   'rich_text': [{'type': 'text',
     'text': {'content': '2009', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': '2009',
     'href': None}]},
  'Author': {'id': 'MouU', 'type': 'rich_text', 'rich_text': []},
  'doi': {'id': 'ZIRT',
   'type': 'rich_text',
   'rich_text': [{'type': 'text',
     'text': {'content': '10.1080/15213260903287259', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
  